# Excel

Mathematics is not just about *getting* a solution, but also about *communicating* it: clearly, precisely, in a manner and using a medium that your audience can understand.

When communicating a large amount of data, particularly to non-specialists, there is one tool that is widely known and used: Microsoft Excel. Whether you like it or loath it, at some point you'll probably have to use it.

Whilst you *can* program in Excel, here we'll focus on using Excel to communicate the results of other programs, or as a source of data.

## Middle-square method

We're going to generate some "random" numbers using Python. We'll then use Excel to see just how random they are. So our three step process is to

1. Write a Python code implementing the [Middle-square method](https://en.wikipedia.org/wiki/Middle-square_method) to generate random numbers;
2. Generate a lot of numbers and save them as an Excel workbook;
3. Analyze the results in Excel.

The Middle-square method takes a *seed*, which is an integer, with $n$ digits ($n$ must be even): for example, $23$ with $2$ digits. It squares the seed to get a number with $2 n$ digits: so $23^2 = 0529$ with $4$ digits. If necessary (as in this example), we "pad" the square with zeros to make the result have $2 n$ digits. Our new "random" number is found by taking the *middle* $n$ digits of the square: so the middle $2$ digits of $0529$ are $52$. To get the next "random" number we repeat the process: $52^2 = 2704 \to 70$, $70^2 = 4900 \to 90$, and so on.

Let's write a Python function that takes a seed and generates $N$ random numbers.

In [1]:
def middle_square(seed, N):
    """
    The Middle-square method for generating pseudo-random numbers.
    
    Parameters
    ----------
    
    seed : int
        The seed for the random number generation, n digits long
    N : int
        The number of random numbers to generate
        
    Returns
    -------
    
    random_list : list of int
        The random numbers: each will be an integer with n digits
    """
    
    n = len(str(seed))
    assert n%2 == 0, "n must be even"
    random_list = []
    previous_rn = seed
    for i in range(N):
        square = str(previous_rn**2)
        digits_square = len(square)
        square_padded = (2*n - digits_square) * '0' + square
        assert len(square_padded) == 2*n
        new_rn = int(square_padded[n-n//2:n+n//2])
        random_list.append(new_rn)
        previous_rn = new_rn
    return random_list

We can now test this on our simple seed, $23$.

In [2]:
print(middle_square(23, 10))

[52, 70, 90, 10, 10, 10, 10, 10, 10, 10]


We see this does a terrible job given this seed: it produces 4 numbers and then just repeats the last in a cycle. Does this always happen? We can use `set` to get the unique entries in the list, to count how well/badly it does:

In [3]:
for i in range(20, 40):
    print(i, len(set(middle_square(i, 15))))

20 2
21 8
22 4
23 4
24 2
25 5
26 5
27 5
28 5
29 4
30 2
31 10
32 2
33 4
34 6
35 5
36 5
37 6
38 8
39 4


Only one of these seeds produces more than a single-digit set of "random" numbers! Very poor.

However, life does improve as we increase the length of the seed:

In [4]:
print(middle_square(4269, 10))

[2243, 310, 961, 9235, 2852, 1339, 7929, 8690, 5161, 6359]


In [5]:
print(len(set(middle_square(4269,100))))

61


In [6]:
print(middle_square(41236797, 10))

[47342681, 32944426, 33520446, 62030003, 72127218, 33557641, 11526948, 87053019, 22811701, 37370251]


In [7]:
print(len(set(middle_square(41236797,10000))))

2364


It would be interesting to analyse how the length of the sequence of "random" numbers increases as we increase the number of seed digits. One way of doing this analysis would be to generate the data in Python and create the plots and analysis in Excel.

## Excel

Using Python and our function above we can rapidly generate a large amount of data. We do not want to copy this into Excel by hand, or by copy-and-paste: both are error prone and tedious. Instead we want the computer to do the work.

We will use the `xlwt` and `xlrd` packages to make Python interact with Excel. We will start by writing some simple data out; then, after some Excel manipulations, we will read it back in to cross-check.

### Writing data

An Excel file is referred to as a *workbook*, which contains *sheets*. To create an Excel file, we create a workbook:

In [8]:
import xlwt
workbook = xlwt.Workbook()

In this book, we then add a sheet, which we will give the name `Data`:

In [9]:
sheet1 = workbook.add_sheet('Data')

We now want to add some data to the sheet. Let us take the initial list of "random" numbers that used the seed $23$:

In [10]:
data_list = middle_square(23, 10)

We will then `write` the data into the sheet. The `write` function takes the row and column number of the cell, and the data to put there:

In [11]:
for row, data in enumerate(data_list):
    sheet1.write(row, 0, data)

Finally, we save the resulting file:

In [12]:
workbook.save('initial_seed.xls')

You should now see the file `initial_seed.xls`: when you open this in Excel, it contains the expected data.

### Reading data

The short Excel file contains the random list in column `A`, rows `1`-`10`. Inside Excel, we can do some simple analysis. Here we will compute the mean by entering the formula `=AVERAGE(A1:A10)` in cell `B1`: the result should be $28.2$. Save the resulting spreadsheet as a new file, `initial_seed_analysed.xls`.

Now we want to read the data in from the new file. In this case we know the data, but this is a useful method for dealing with large datasets that may be provided in Excel files.

First we open the workbook:

In [13]:
import xlrd
workbook_new = xlrd.open_workbook('initial_seed_analysed.xls')

There may, in general, be many sheets. Here we find them all (there is only one), and work with the first:

In [14]:
all_sheets = workbook_new.sheets()
sheet1 = all_sheets[0]

We assume that we know that the data is in the first column, and the mean at the top of the second. We find out how much data there is by checking the number of rows:

In [15]:
data_length = sheet1.nrows
print(data_length)

10


We then read all the data into a new list:

In [16]:
data_list_analysed = []
for row in range(data_length):
    data_list_analysed.append(sheet1.cell(row, 0).value)
print(data_list_analysed)

[52.0, 70.0, 90.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]


Note that the results are floating point numbers, not the integers from the original list.

We also read in the mean of the data:

In [17]:
mean_excel = sheet1.cell(0, 1).value
print(mean_excel)

28.2


We have the expected results: by using a consistent data layout, and the `xlrd` variables `nrows` (as above), `ncols`, and `nsheets`, we can generalize this approach to read very large data sets.

# Exercise : Random seed length effects

## Exercise 1

Using the code above, find out how many *unique* random numbers are generated by the Middle-square method, for random seeds between the smallest $n$-digit number (e.g., $10$), and that number plus $10$. Do this for $n = 2, 4, 6, 8, 10$. This means the random seeds should be in $[10, 20]$, $[1000, 1010]$, $[100000, 100010]$, and so on.

**Note**: for these seeds the maximum length of the sequence is less that $10^5$.

In [18]:
for ndigits in range(1,10,2):
    for seed in range(10**ndigits, 10**ndigits+11):
        print(seed, len(set(middle_square(seed, 100000))))

10 1
11 9
12 8
13 7
14 7
15 5
16 6
17 6
18 3
19 6
20 2
1000 1
1001 3
1002 4
1003 5
1004 6
1005 1
1006 44
1007 14
1008 24
1009 10
1010 14
100000 1
100001 4
100002 4
100003 5
100004 6
100005 1
100006 657
100007 624
100008 176
100009 377
100010 22
10000000 1
10000001 4
10000002 5
10000003 6
10000004 7
10000005 1
10000006 834
10000007 6482
10000008 6492
10000009 189
10000010 102
1000000000 1
1000000001 4
1000000002 5
1000000003 6
1000000004 7
1000000005 1
1000000006 70616
1000000007 41095
1000000008 21526
1000000009 41
1000000010 503


## Exercise 2

Create an Excel workbook containing the data. The first row should contain the number of digits. The next eleven rows should contain the length of the sequence.

In [19]:
import xlwt
workbook = xlwt.Workbook()
sheet1 = workbook.add_sheet('Data')
for column, ndigits in enumerate(range(1,10,2)):
    sheet1.write(0, column, ndigits+1)
    for row, seed in enumerate(range(10**ndigits, 10**ndigits+11)):
        sequence_length = len(set(middle_square(seed, 100000)))
        sheet1.write(row+1, column, sequence_length)
workbook.save('Excel_Exercise1.xls')

## Exercise 3

Open the workbook in Excel. Add a second sheet called `Analysis`. The first row should contain the number of digits, as in the first row on the `Data` sheet. The second row should contain the mean of the sequence lengths for that corresponding number of digits. Save the result as a new Excel file.

## Exercise 4

Read the data from the second sheet into Python and print it to the screen in the form (for example)

`For 2 digits the average sequence length is 5.45.`

Can you see a pattern?

In [20]:
import xlrd
workbook = xlrd.open_workbook('Excel_Exercise1_Analysis.xls')
all_sheets = workbook.sheets()
sheet2 = all_sheets[1]
for column in range(sheet2.ncols):
    ndigits = sheet2.cell(0, column).value
    average_length = sheet2.cell(1, column).value
    print("For {} digits the average sequence length is {:.2f}".format(int(ndigits), average_length))

For 2 digits the average sequence length is 5.45
For 4 digits the average sequence length is 11.45
For 6 digits the average sequence length is 170.64
For 8 digits the average sequence length is 1283.91
For 10 digits the average sequence length is 12164.09
